In [ ]:
pip install torch==1.4.0

In [ ]:
!pip3 install transformers==4.0.0

In [1]:
import pandas as pd


In [ ]:
file = open("prigov.txt")

prigov_texts = file.read()
file.close()

prigov_t=prigov_texts.replace("\n"," \n ").split("*")

for i in range(1,len(prigov_t)):
  prigov_t[i]=prigov_t[i][3:]

stihi = pd.DataFrame({'text': prigov_t
                   })

In [ ]:
stihi

,text
0,Соловьи солдат тревожат \n То им пятки пощекоч...
1,Когда она приходит в сад \n Почти растений не ...
2,Две старушки на пенечке \n Примостились на опу...
3,Я ответил: Милый мой \n Кто же знает – за спин...
4,"Немножко кушают, немножко \n О счастье думают,..."
...,...
549,Я чаю заварю \n Задумаюсь над зверем \n И к св...
550,В ночи позвякивают петли \n Дверные \n И кто-т...
551,Шорох дождя по кровле \n Да поскрипыванье двер...
552,И речь ручья так непонятна \n И дерева не обой...


In [ ]:
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments 
from transformers import  GPT2LMHeadModel
from transformers import pipeline
import re
import torch 
print(torch.__version__)

In [ ]:
def build_text_files(txts, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for txt in txts:
        summary = txt.strip()
        #summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [ ]:
train, test = train_test_split(prigov_t, test_size=0.15)

In [ ]:
build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')

In [ ]:
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [ ]:
def load_dataset(train_path,test_path,tokenizer):

    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=16)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=16)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,

    )
    return train_dataset,test_dataset,data_collator

In [ ]:
train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:44: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")

In [ ]:
training_args = TrainingArguments(
    output_dir="./rugpt3", 
    overwrite_output_dir=True, 
    num_train_epochs=3, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=64,  
    eval_steps = 400, 
    #learning_rate=4e-5,
    save_steps=8000, 
    warmup_steps=300
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()
trainer.save_model()

Step,Training Loss


In [ ]:
text_generation = pipeline('text-generation', model='./rugpt3', tokenizer='sberbank-ai/rugpt3medium_based_on_gpt2', config={'max_length':1000, 'temperature': 1.2})


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
 from pprint import pprint

In [ ]:
text = "Любовь"

In [ ]:
result = text_generation(text, max_length=70, top_k=50, num_return_sequences=1, top_p=0.92, do_sample=True, temperature=1.2)[0]['generated_text']
pprint(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


('Любовь у вас была – вот что \n'
 ' Нет, я ее  Промежду нас и дальше  А как он меня ненавидел  Что же это – ни '
 'дня ему не спалось \n'
 ' И вот в окошко поглядел он \n'
 ' Как я вхожу – как будто смерть пришла \n'
 ' А рядом стоял он \n'
 ' Как его мне так и жаль было! – \n'
 ' Да')
